# Train with LightGBM in an Interactive Run

description: use AML and mlflow to track interactive experimentation in the cloud 

In [ ]:
!pip install --upgrade lightgbm scikit-learn

## Setup cloud tracking

In [ ]:
from azureml.core import Workspace
import mlflow

ws = Workspace.from_config()
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())
mlflow.set_experiment("lightgbm-iris-local-example")

## Load data

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import lightgbm as lgb

# prepare train and test data
iris = datasets.load_iris()
X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
train_set = lgb.Dataset(X_train, label=y_train)

## Train model

In [ ]:
from sklearn.metrics import accuracy_score, log_loss

# set training parameters
params = {
    "objective": "multiclass",
    "num_class": 3,
    "learning_rate": 0.1,
    "metric": "multi_logloss",
    "colsample_bytree": 1.0,
    "subsample": 1.0,
    "seed": 42,
}

# start run
run = mlflow.start_run()

# enable automatic logging
mlflow.lightgbm.autolog()

# train the lightgbm model
model = lgb.train(params, train_set, num_boost_round=10,)

# evaluate model
y_proba = model.predict(X_test)
y_pred = y_proba.argmax(axis=1)
loss = log_loss(y_test, y_proba)
acc = accuracy_score(y_test, y_pred)

# log metrics from evaluation
mlflow.log_metric("log_loss", loss)
mlflow.log_metric("accuracy", acc)

# end run
mlflow.end_run()